<div class='alert alert-warning'>

SciPy's interactive examples with Jupyterlite are experimental and may not always work as expected. Execution of cells containing imports may result in large downloads (up to 60MB of content for the first import from SciPy). Load times when importing from SciPy may take roughly 10-20 seconds. If you notice any problems, feel free to open an [issue](https://github.com/scipy/scipy/issues/new/choose).

</div>

The following plot illustrates the envelope of a signal with variable frequency and
a low-frequency drift. To separate the drift from the envelope, a 4 Hz highpass
filter is used. The low-pass residuum of the input bandpass filter is utilized to
determine an asymmetric upper and lower bound to enclose the signal. Due to the
smoothness of the resulting envelope, it is down-sampled from 500 to 40 samples.
Note that the instantaneous amplitude ``a_x`` and the computed envelope ``x_env``
are not perfectly identical. This is due to the signal not being perfectly periodic
as well as the existence of some spectral overlapping of ``x_carrier`` and
``x_drift``. Hence, they cannot be completely separated by a bandpass filter.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal.windows import gaussian
from scipy.signal import envelope

n, n_out = 500, 40  # number of signal samples and envelope samples
T = 2 / n  # sampling interval for 2 s duration
t = np.arange(n) * T  # time stamps
a_x = gaussian(len(t), 0.4/T)  # instantaneous amplitude
phi_x = 30*np.pi*t + 35*np.cos(2*np.pi*0.25*t)  # instantaneous phase
x_carrier = a_x * np.cos(phi_x)
x_drift = 0.3 * gaussian(len(t), 0.4/T)  # drift
x = x_carrier + x_drift

bp_in = (int(4 * (n*T)), None)  # 4 Hz highpass input filter
x_env, x_res = envelope(x, bp_in, n_out=n_out)
t_out = np.arange(n_out) * (n / n_out) * T

fg0, ax0 = plt.subplots(1, 1, tight_layout=True)
ax0.set_title(r"$4\,$Hz Highpass Envelope of Drifting Signal")
ax0.set(xlabel="Time in seconds", xlim=(0, n*T), ylabel="Amplitude")
ax0.plot(t, x, 'C0-', alpha=0.5, label="Signal")
ax0.plot(t, x_drift, 'C2--', alpha=0.25, label="Drift")
ax0.plot(t_out, x_res+x_env, 'C1.-', alpha=0.5, label="Envelope")
ax0.plot(t_out, x_res-x_env, 'C1.-', alpha=0.5, label=None)
ax0.grid(True)
ax0.legend()
plt.show()

The second example provides a geometric envelope interpretation of complex-valued
signals: The following two plots show the complex-valued signal as a blue
3d-trajectory and the envelope as an orange round tube with varying diameter, i.e.,
as $|a(t)| \exp(j\rho(t))$, with $\rho(t)\in[-\pi,\pi]$. Also, the
projection into the 2d real and imaginary coordinate planes of trajectory and tube
is depicted. Every point of the complex-valued signal touches the tube's surface.

The left plot shows an analytic signal, i.e, the phase difference between
imaginary and real part is always 90 degrees, resulting in a spiraling trajectory.
It can be seen that in this case the real part has also the expected envelope,
i.e., representing the absolute value of the instantaneous amplitude.

The right plot shows the real part of that analytic signal being interpreted
as a complex-vauled signal, i.e., having zero imaginary part. There the resulting
envelope is not as smooth as in the analytic case and the instantaneous amplitude
in the real plane is not recovered. If ``z_re`` had been passed as a real-valued
signal, i.e., as ``z_re = z.real`` instead of ``z_re = z.real + 0j``, the result
would have been identical to the left plot. The reason for this is that real-valued
signals are interpreted as being the real part of a complex-valued analytic signal.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal.windows import gaussian
from scipy.signal import envelope

n, T = 1000, 1/1000  # number of samples and sampling interval
t = np.arange(n) * T  # time stamps for 1 s duration
f_c = 3  # Carrier frequency for signal
z = gaussian(len(t), 0.3/T) * np.exp(2j*np.pi*f_c*t)  # analytic signal
z_re = z.real + 0j  # complex signal with zero imaginary part

e_a, e_r = (envelope(z_, (None, None), residual=None) for z_ in (z, z_re))

# Generate grids to visualize envelopes as 2d and 3d surfaces:
E2d_t, E2_amp = np.meshgrid(t, [-1, 1])
E2d_1 = np.ones_like(E2_amp)
E3d_t, E3d_phi = np.meshgrid(t, np.linspace(-np.pi, np.pi, 300))
ma = 1.8  # maximum axis values in real and imaginary direction

fg0 = plt.figure(figsize=(6.2, 4.))
ax00 = fg0.add_subplot(1, 2, 1, projection='3d')
ax01 = fg0.add_subplot(1, 2, 2, projection='3d', sharex=ax00,
                       sharey=ax00, sharez=ax00)
ax00.set_title("Analytic Signal")
ax00.set(xlim=(0, 1), ylim=(-ma, ma), zlim=(-ma, ma))
ax01.set_title("Real-valued Signal")
for z_, e_, ax_ in zip((z, z.real), (e_a, e_r), (ax00, ax01)):
    ax_.set(xlabel="Time $t$", ylabel="Real Amp. $x(t)$",
            zlabel="Imag. Amp. $y(t)$")
    ax_.plot(t, z_.real, 'C0-', zs=-ma, zdir='z', alpha=0.5, label="Real")
    ax_.plot_surface(E2d_t, e_*E2_amp, -ma*E2d_1, color='C1', alpha=0.25)
    ax_.plot(t, z_.imag, 'C0-', zs=+ma, zdir='y', alpha=0.5, label="Imag.")
    ax_.plot_surface(E2d_t, ma*E2d_1, e_*E2_amp, color='C1', alpha=0.25)
    ax_.plot(t, z_.real, z_.imag, 'C0-', label="Signal")
    ax_.plot_surface(E3d_t, e_*np.cos(E3d_phi), e_*np.sin(E3d_phi),
                     color='C1', alpha=0.5, shade=True, label="Envelope")
    ax_.view_init(elev=22.7, azim=-114.3)
fg0.subplots_adjust(left=0.08, right=0.97, wspace=0.15)
plt.show()